In [554]:
import enum
import functools
import numpy as np
import pandas as pd
from scipy import stats
from scipy import linalg
from typing import Callable, List, NamedTuple, Sequence, Tuple

BETA = np.array([0., 0.5], dtype=np.float64)
DESIGN_CLUSTERS = {
    'I': [[7, 10, 13, 16]],
    'II': [[7, 10, 13], [7, 10, 16], [7, 13, 16], [10, 13, 16]],
}
NUM_CLUSTERS = [15, 30, 60]
WITHIN_CLUSTER_CORRELATIONS = [0.5, 0.9]

CorrelationStructure = enum.Enum(
    'CorrelationStructure',
    'NONE EXCHANGEABLE EXPONENTIAL')

EstimationMethod = enum.Enum(
    'EstimationMethod',
    'OLS QL Sandwich')

In [640]:
class Experiment(NamedTuple('Experiment', [
    ('beta', np.array),
    ('error_variance', float),
    ('num_clusters', Sequence[Tuple[np.array, np.array]]),
    ('clusters', Sequence[np.array]),
    ('within_cluster_correlation', float),
    ('within_cluster_correlation_structure', CorrelationStructure),
])):
    """Encapsulates parameters for the data generating mechanism."""
    
    def sample_clusters(self) -> List[Tuple[np.array, np.array]]:
        return [self._sample_cluster() for _ in range(self.num_clusters)]
    
    def _sample_cluster(self) -> Tuple[np.array, np.array]:
        covariates = self._sample_cluster_covariates()
        covariates = np.column_stack((np.ones(len(covariates)), covariates))
        covariance = self._make_within_cluster_covariance(len(covariates))
        response = stats.multivariate_normal(
            mean=np.matmul(covariates, self.beta), cov=covariance).rvs()
        return covariates, response
    
    def _sample_cluster_covariates(self) -> np.array:
        return self.clusters[np.random.choice(len(self.clusters))]
    
    def _make_within_cluster_covariance(self, cluster_size):
        correlation = np.eye(cluster_size)
        if self.within_cluster_correlation_structure == CorrelationStructure.EXCHANGEABLE:
            correlation[correlation == 0] = self.within_cluster_correlation
        elif self.within_cluster_correlation_structure == CorrelationStructure.EXPONENTIAL:
            for i in range(cluster_size):
                for j in range(i + 1, cluster_size):
                    correlation[i, j] = correlation[j, i] = np.power(
                        self.within_cluster_correlation, np.abs(j - i))
        return self.error_variance*correlation
        
    @classmethod
    def from_template(
        cls,
        clusters,
        num_clusters,
        within_cluster_correlation,
        within_cluster_correlation_structure) -> 'Experiment':
        assert len(set([len(cluster) for cluster in clusters])) == 1,\
               'Clusters must be the same size.'
        
        return cls(beta=BETA,
                   clusters=clusters,
                   error_variance=1.,
                   num_clusters=num_clusters,
                   within_cluster_correlation=within_cluster_correlation,
                   within_cluster_correlation_structure=within_cluster_correlation_structure)

In [652]:
def estimate_rho(epsilon_hat):
    covariance = np.outer(epsilon_hat, epsilon_hat)    
    rho_exchangeable = 0.
    rho_exponential = 0.
    for i in range(len(covariance)):
        for j in range(i + 1, len(covariance[i])):
            rho_exchangeable += covariance[i, j]
            if j - i == 1:
                rho_exponential += covariance[i, j]
    cluster_pairwise_count = (covariance.shape[0] - 1)*(covariance.shape[1] - 2)
    rho_exchangeable /= cluster_pairwise_count
    rho_exponential /= covariance.shape[0] - 1
    return rho_exchangeable, rho_exponential
    
def run_experiment(experiment):
    clusters = experiment.sample_clusters()
    X = np.vstack([X for X, _ in clusters])
    y = np.hstack([y for _, y in clusters])
    
    gram_matrix = X.T.dot(X)
    gram_matrix_inv = linalg.cho_solve(linalg.cho_factor(gram_matrix), np.eye(len(gram_matrix)))
    
    beta_hat = gram_matrix_inv.dot(X.T).dot(y)
    
    epsilon_hat = y - X.dot(beta_hat)
    sigma_2_hat = epsilon_hat.dot(epsilon_hat)/(len(y) - len(beta_hat))
    

    sandwich_variance = gram_matrix_inv.dot(
        X.T.dot(np.diag(np.square(epsilon_hat))).dot(X)).dot(gram_matrix_inv)
    
    rho_exchangeable = 0.
    rho_exponential = 0.        
    for cluster_X, cluster_y in clusters:
        cluster_rho_exchangeable, cluster_rho_exponential = estimate_rho(
            cluster_y - cluster_X.dot(beta_hat))
        rho_exchangeable += cluster_rho_exchangeable
        rho_exponential += cluster_rho_exponential
                                              
    rho_exchangeable /= len(clusters)
    rho_exponential /= len(clusters)
    print(rho_exchangeable)
    print(rho_exponential)
        
    return {
        EstimationMethod.OLS.name: {
            CorrelationStructure.NONE.name: np.sqrt(gram_matrix_inv[1, 1]),
            CorrelationStructure.EXCHANGEABLE.name: 0.,
            CorrelationStructure.EXPONENTIAL.name: 0.,
        },
        EstimationMethod.QL.name: {
            CorrelationStructure.NONE.name: np.sqrt(gram_matrix_inv[1, 1]*sigma_2_hat),
            CorrelationStructure.EXCHANGEABLE.name: 0.5,
            CorrelationStructure.EXPONENTIAL.name: 0.,            
        },
        EstimationMethod.Sandwich.name: {
            CorrelationStructure.NONE.name: np.sqrt(sandwich_variance[1, 1]),
            CorrelationStructure.EXCHANGEABLE.name: 0.,
            CorrelationStructure.EXPONENTIAL.name: 0.,
        },
    }

def run_experiments(experiment, num_trials):
    def _merge_results(acc, result):
        if type(acc) == dict:        
            return {key: _merge_results(value, result[key]) for key, value in acc.items()}   
        elif type(acc) in {np.float64, float, int}:
            return acc + result        
        raise ValueError('Unknown type: {}'.format(type(acc).__name__))
        
    def _divide_results(results, d):
        if type(results) == dict:
            return {key: _divide_results(value, d) for key, value in results.items()}
        return results/d
                    
    results = [run_experiment(experiment) for _ in range(num_trials)]
    results = functools.reduce(_merge_results, results)
    return _divide_results(results, num_trials)

experiment = Experiment.from_template(
    DESIGN_CLUSTERS['I'],
    #NUM_CLUSTERS[2],
    1000,
    WITHIN_CLUSTER_CORRELATIONS[1],
    CorrelationStructure.EXPONENTIAL)

#tmp = run_experiments(experiment, 128)
tmp = run_experiment(experiment)
#experiment.sample_clusters()
tmp

0.8140454723739351
0.8773015400721659


{'OLS': {'EXCHANGEABLE': 0.0,
  'EXPONENTIAL': 0.0,
  'NONE': 0.004714045207910317},
 'QL': {'EXCHANGEABLE': 0.5, 'EXPONENTIAL': 0.0, 'NONE': 0.004662787355928457},
 'Sandwich': {'EXCHANGEABLE': 0.0,
  'EXPONENTIAL': 0.0,
  'NONE': 0.0046641498322187724}}

In [322]:
pd.DataFrame.from_dict(tmp, orient='index').stack()

OLS       NONE            0.9
          EXCHANGEABLE    0.5
          EXPONENTIAL     0.0
QL        NONE            0.0
          EXCHANGEABLE    0.0
          EXPONENTIAL     0.0
Sandwich  NONE            0.0
          EXCHANGEABLE    0.0
          EXPONENTIAL     0.0
dtype: float64

In [390]:
def index_experiment(experiment):    
    return (experiment.num_clusters,
            [k for k, v in DESIGN_CLUSTERS.items() if experiment.clusters == v][0],
            experiment.within_cluster_correlation_structure.name,
            experiment.within_cluster_correlation)

simulation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [NUM_CLUSTERS, DESIGN_CLUSTERS.keys(),
         [CorrelationStructure.EXCHANGEABLE.name, CorrelationStructure.EXPONENTIAL.name],
         WITHIN_CLUSTER_CORRELATIONS],
        names=['$n$', 'Design', 'Correlation structure', 'Correlation']),
    columns=pd.MultiIndex.from_product(
        [[value.name for value in EstimationMethod],
         [value.name for value in CorrelationStructure]],
        names=['Estimator', 'Assumed correlation']
    ))

simulation_results.loc[index_experiment(experiment)] = (
    pd.DataFrame.from_dict(tmp, orient='index').stack())
simulation_results.to_dict()

{('OLS', 'EXCHANGEABLE'): {(15, 'I', 'EXCHANGEABLE', 0.5): nan,
  (15, 'I', 'EXCHANGEABLE', 0.9): nan,
  (15, 'I', 'EXPONENTIAL', 0.5): nan,
  (15, 'I', 'EXPONENTIAL', 0.9): nan,
  (15, 'II', 'EXCHANGEABLE', 0.5): nan,
  (15, 'II', 'EXCHANGEABLE', 0.9): nan,
  (15, 'II', 'EXPONENTIAL', 0.5): nan,
  (15, 'II', 'EXPONENTIAL', 0.9): nan,
  (30, 'I', 'EXCHANGEABLE', 0.5): nan,
  (30, 'I', 'EXCHANGEABLE', 0.9): nan,
  (30, 'I', 'EXPONENTIAL', 0.5): nan,
  (30, 'I', 'EXPONENTIAL', 0.9): nan,
  (30, 'II', 'EXCHANGEABLE', 0.5): nan,
  (30, 'II', 'EXCHANGEABLE', 0.9): nan,
  (30, 'II', 'EXPONENTIAL', 0.5): nan,
  (30, 'II', 'EXPONENTIAL', 0.9): nan,
  (60, 'I', 'EXCHANGEABLE', 0.5): nan,
  (60, 'I', 'EXCHANGEABLE', 0.9): nan,
  (60, 'I', 'EXPONENTIAL', 0.5): nan,
  (60, 'I', 'EXPONENTIAL', 0.9): nan,
  (60, 'II', 'EXCHANGEABLE', 0.5): nan,
  (60, 'II', 'EXCHANGEABLE', 0.9): nan,
  (60, 'II', 'EXPONENTIAL', 0.5): 0.0,
  (60, 'II', 'EXPONENTIAL', 0.9): nan},
 ('OLS', 'EXPONENTIAL'): {(15, 'I', 'E

<CorrelationStructure.NONE: 1>

In [177]:
np.random.choice([[7, 9], [2, 3], [1, 2]])

ValueError: a must be 1-dimensional

In [42]:
linalg.cholesky(tmp).T.dot(linalg.cholesky(tmp))

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [55]:
linalg.inv(tmp)

array([[ 1.6, -0.4, -0.4, -0.4],
       [-0.4,  1.6, -0.4, -0.4],
       [-0.4, -0.4,  1.6, -0.4],
       [-0.4, -0.4, -0.4,  1.6]])

In [52]:
np.sqrt(3)/6

0.28867513459481287